In [1]:
import numpy as np
import tensorflow as tf

In [2]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(28, 28), name='input'),
    tf.keras.layers.LSTM(20, time_major=False, return_sequences=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax, name='output')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 28, 20)            3920      
_________________________________________________________________
flatten (Flatten)            (None, 560)               0         
_________________________________________________________________
output (Dense)               (None, 10)                5610      
Total params: 9,530
Trainable params: 9,530
Non-trainable params: 0
_________________________________________________________________


In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

# Change this to True if you want to test the flow rapidly.
# Train with a small dataset and only 1 epoch. The model will work poorly
# but this provides a fast way to test if the conversion works end to end.
_FAST_TRAINING = False
_EPOCHS = 5
if _FAST_TRAINING:
  _EPOCHS = 1
  _TRAINING_DATA_COUNT = 1000
  x_train = x_train[:_TRAINING_DATA_COUNT]
  y_train = y_train[:_TRAINING_DATA_COUNT]

model.fit(x_train, y_train, epochs=_EPOCHS)
model.evaluate(x_test, y_test, verbose=0)

11501568/11490434 [==============================] - 7s 1us/step
Epoch 1/5
1875/1875 [==============================] - 41s 10ms/step - loss: 0.3423 - accuracy: 0.8979
Epoch 2/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.1254 - accuracy: 0.9620
Epoch 3/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0906 - accuracy: 0.9722
Epoch 4/5
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0724 - accuracy: 0.9778
Epoch 5/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0611 - accuracy: 0.9810


[0.05477413162589073, 0.9807999730110168]

In [4]:
run_model = tf.function(lambda x: model(x))
# This is important, let's fix the input size.
BATCH_SIZE = 1
STEPS = 28
INPUT_SIZE = 28
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([BATCH_SIZE, STEPS, INPUT_SIZE], model.inputs[0].dtype))

# model directory.
MODEL_DIR = "keras_lstm"
model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: keras_lstm\assets


INFO:tensorflow:Assets written to: keras_lstm\assets


In [6]:
def representative_dataset():
    data = np.random.random((1,28,28))
    yield [data.astype(np.float32)]
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# # converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
# #                                        tf.lite.OpsSet.SELECT_TF_OPS]
# # converter.experimental_new_converter = True
# # Enforce integer only quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_dataset
model_tflite = converter.convert()

# Save the model to disk
# open(f_modele_lite_path + prefix +'quantized.tflite', "wb").write(model_tflite)